In [ ]:
!pip install fasttext

import fasttext
import pandas as pd
import re
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

train_df = pd.read_parquet("/kaggle/input/wsdm-cup-multilingual-chatbot-arena/train.parquet")

train_df['texts'] = train_df['prompt'] + " " + train_df['response_a'] + " " + train_df['response_b']
train_df['labels'] = train_df['winner'].map(lambda x: 0 if x == 'model_a' else 1)

df = train_df[["texts", "labels"]]

train_df, test_df = train_test_split(df, test_size=0.2, random_state=1, stratify=df["labels"])

def convert_to_fasttext_format(df, output_file):
    with open(output_file, "w") as f:
        for _, row in df.iterrows():
            text = re.sub(r"\s+", " ", row["texts"]).strip()
            f.write(f"__label__{row['labels']} {text}\n")

convert_to_fasttext_format(train_df, "train.txt")
convert_to_fasttext_format(test_df, "test.txt")

model = fasttext.train_supervised("train.txt", epoch=100, lr=0.001, wordNgrams=3, loss='softmax')

def evaluate(model, test_file):
    true_labels, texts = [], []
    with open(test_file, "r") as f:
        for line in f:
            parts = line.strip().split(" ")
            label = int(parts[0].replace("__label__", ""))
            text = " ".join(parts[1:])
            true_labels.append(label)
            texts.append(text)
    
    pred_labels = [int(model.predict(text)[0][0].replace("__label__", "")) for text in texts]
    print("Accuracy:", accuracy_score(true_labels, pred_labels))
    print(classification_report(true_labels, pred_labels))

evaluate(model, "test.txt")

Accuracy: 0.505057803468208
              precision    recall  f1-score   support

           0       0.50      0.09      0.15      4792
           1       0.51      0.91      0.65      4896

    accuracy                           0.51      9688
   macro avg       0.50      0.50      0.40      9688
weighted avg       0.50      0.51      0.40      9688

